In [2]:
from other.data.audio_utils import AudioWorker
from other.data.audio_utils import get_files_by_extension

# from other.data.augmentation_utils import augment_volume_gain

openSLR_data_directory, ext = '../datasets/train-clean-100(converted to 8000 sr)', 'flac'
audio_files_paths = get_files_by_extension(openSLR_data_directory, ext=ext)

In [136]:
import torch


def augment_volume_gain(aw, gain_function='random', effect_ratio=(0.2, 1), value_range=(0.05, 1), inverse=0.5):
    functions = ['sin', 'woods']
    if gain_function == "random":
        ind = torch.randint(0, len(functions), (1,)).item()
        gain_function = functions[ind]
    if not isinstance(effect_ratio, (float, int)):
        i_min, i_max = effect_ratio
        mean, std = 0.5, 0.25
        r = torch.clamp(mean + torch.randn(1) * std, 0.0, 1.0).item()
        effect_ratio = i_min + r * (i_max - i_min)

    match gain_function:
        case 'sin':
            gain = torch.sin(torch.linspace(0, 2 * torch.pi, steps=aw.length)) ** (2 * int(25 ** effect_ratio))
        case 'woods':
            a, b, c = effect_ratio, effect_ratio, 0.5
            n = lambda x: 1 / torch.tan(torch.pi * (2 * torch.abs(x - c) / b))
            m = lambda x: torch.exp(n(x))
            v = lambda x: 1 / (1 + m(x) ** (100 ** a))
            fr, to = c - b / 2, c + b / 2
            gain = torch.ones(aw.length)
            s_fr, s_to = int(aw.length * fr), int(aw.length * to)
            gain[s_fr: s_to] = v(torch.linspace(fr, to, steps=s_to - s_fr))
        case _:
            ValueError("Unsupported gain function. Choose 'sin' or 'woods'.")
            return

    was_inversed = torch.rand(1).item() < inverse
    if was_inversed:
        gain = 1 - gain

    min_gain, max_gain = value_range
    gain = min_gain + gain * (max_gain - min_gain)
    aw.wave *= torch.abs(gain.to(aw.wave.device))

    return {"was_inversed": was_inversed, "gain_function": gain_function, "effect_ratio": effect_ratio}

In [137]:
aw = AudioWorker(audio_files_paths[112], 'TEST')
aw.load()
aw.player()

In [153]:
aw.load()
print(*augment_volume_gain(aw, value_range=(0.05, 1)).items(), sep='\n')
aw.player()

('was_inversed', False)
('gain_function', 'woods')
('effect_ratio', 0.4897139310836792)


In [38]:
aw.load()
augment_volume_gain(aw, effect_ratio=0.75, value_range=(0.05, 1))
aw.player()

In [39]:
aw.load()
augment_volume_gain(aw, effect_ratio=0.5, value_range=(0.05, 1))
aw.player()

In [40]:
aw.load()
augment_volume_gain(aw, effect_ratio=0.25, value_range=(0.05, 1))
aw.player()

In [41]:
aw.load()
augment_volume_gain(aw, effect_ratio=0, value_range=(0.05, 1))
aw.player()